# Building a quick and dirty bigram LM

First import some useful libraries.

In [1]:
from collections import Counter

In [2]:
from collections import defaultdict

In [52]:
import numpy as np

Next let's pull in all the data into one long string (note this is not really a great idea from a practical programming perspective).

In [182]:
text = open("hobbit-train.txt").read()

In [183]:
len(text)

521691

Split on white space. 'words' is now just a list of strings.

In [184]:
words = text.split()

In [174]:
len(words)

95940

Let's use a Counter to get the frequencies of the words in the corpus. A Counter is basically a dictionary that keeps track of the frequency of items in a collection. The keys are the items and the values are their frequencies in the collection.

In [185]:
rawfreqs = Counter(words)

In [186]:
len(rawfreqs)

12333

So there are 12333 unique strings in the Hobbit (using whitespace to split on). One of them is surely 'Gandalf'.

In [187]:
rawfreqs["Gandalf"]

99

Keeping track of words and their counts using strings isn't very efficient. So before moving on we'll map known words to unique integer indexes. To do this we'll use the defaultdict flavor of dictionary that takes a method that specifies how to create values for new entries to the index. So in the following, word2index is a dictionary where the keys are strings and the values are the integer index for each string. We'll assign the index based on when the word is encountered as we walk through the text.

In [188]:
word2index = defaultdict(lambda: len(word2index))

And let's make <UNK> the first entry to the index.

In [189]:
UNK = word2index["<UNK>"]

In [190]:
word2index["<UNK>"]

0

In [191]:
UNK

0

Now let's build our actual lexicon. We'll use the heuristic that we can ignore words that only happened once in training.

In [192]:
[word2index[word] for word, freq in rawfreqs.items() if freq > 1]

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

Do that we're down to roughly 5000 words from 12000 in our vocabulary. 

In [193]:
len(word2index)

5209

In [194]:
len(rawfreqs)

12333

Now we need to freeze the lexicon. And, in particular, we want to map new words to UNK and not add them to the lexicon.  So we need to change the default method for word2index.

In [195]:
word2index = defaultdict(lambda: UNK, word2index)

In [196]:
word2index["Gandalf"]

363

In [197]:
word2index["Ralphie"]

0

For a bigram model we need bigrams. So back to words.

We can get all the bigrams in the corpus by going back to the words and stepping through it a pair at a time.

In [198]:
for i in range(1, len(words)): print(words[i-1], words[i])

THE HOBBIT
HOBBIT OR
OR THERE
THERE AND
AND BACK
BACK AGAIN
AGAIN J.R.R.
J.R.R. TOLKIEN
TOLKIEN Chapter
Chapter I
I AN
AN UNEXPECTED
UNEXPECTED PARTY
PARTY In
In a
a hole
hole in
in the
the ground
ground there
there lived
lived a
a hobbit.
hobbit. Not
Not a
a nasty,
nasty, dirty,
dirty, wet
wet hole,
hole, filled
filled with
with the
the ends
ends of
of worms
worms and
and an
an oozy
oozy smell,
smell, nor
nor yet
yet a
a dry,
dry, bare,
bare, sandy
sandy hole
hole with
with nothing
nothing in
in it
it to
to sit
sit down
down on
on or
or to
to eat:
eat: it
it was
was a
a hobbit-hole,
hobbit-hole, and
and that
that means
means comfort.
comfort. It
It had
had a
a perfectly
perfectly round
round door
door like
like a
a porthole,
porthole, painted
painted green,
green, with
with a
a shiny
shiny yellow
yellow brass
brass knob
knob in
in the
the exact
exact middle.
middle. The
The door
door opened
opened on
on to
to a
a tube-shaped
tube-shaped hall
hall like
like a
a tunnel:
tunnel: a
a very

a quarter
quarter of
of what
what I
I have
have heard
heard about
about him,
him, and
and I
I have
have only
only heard
heard very
very little
little of
of all
all there
there is
is to
to hear,
hear, you
you would
would be
be prepared
prepared for
for any
any sort
sort of
of remarkable
remarkable tale.
tale. Tales
Tales and
and adventures
adventures sprouted
sprouted up
up all
all over
over the
the place
place wherever
wherever he
he went,
went, in
in the
the most
most extraordinary
extraordinary fashion.
fashion. He
He had
had not
not been
been down
down that
that way
way under
under The
The Hill
Hill for
for ages
ages and
and ages,
ages, not
not since
since his
his friend
friend the
the Old
Old Took
Took died,
died, in
in fact,
fact, and
and the
the hobbits
hobbits had
had almost
almost forgotten
forgotten what
what he
he looked
looked like.
like. He
He had
had been
been away
away over
over The
The Hill
Hill and
and across
across The
The Water
Water on
on businesses
businesses of
of 

up. He
He hung
hung his
his red
red one
one next
next to
to it,
it, and
and “Balin
“Balin at
at your
your service!”
service!” he
he said
said with
with his
his hand
hand on
on his
his breast.
breast. “Thank
“Thank you!”
you!” said
said Bilbo
Bilbo with
with a
a gasp.
gasp. It
It was
was not
not the
the correct
correct thing
thing to
to say,
say, but
but they
they have
have begun
begun to
to arrive
arrive had
had flustered
flustered him
him badly.
badly. He
He liked
liked visitors,
visitors, but
but he
he liked
liked to
to know
know them
them before
before they
they arrived,
arrived, and
and he
he preferred
preferred to
to ask
ask them
them himself.
himself. He
He had
had a
a horrible
horrible thought
thought that
that the
the cakes
cakes might
might run
run short,
short, and
and then
then he
he —
— as
as the
the host:
host: he
he knew
knew his
his duty
duty and
and stuck
stuck to
to it
it however
however painful
painful —
— he
he might
might have
have to
to go
go without.
without. “Com

the kitchen
kitchen trying
trying to
to see
see what
what they
they were
were doing.
doing. Then
Then they
they went
went back,
back, and
and found
found Thorin
Thorin with
with his
his feet
feet on
on the
the fender
fender smoking
smoking a
a pipe.
pipe. He
He was
was blowing
blowing the
the most
most enormous
enormous smoke-rings,
smoke-rings, and
and wherever
wherever he
he told
told one
one to
to go,
go, it
it went
went —
— up
up the
the chimney,
chimney, or
or behind
behind the
the clock
clock on
on the
the mantelpiece,
mantelpiece, or
or under
under the
the table,
table, or
or round
round and
and round
round the
the ceiling;
ceiling; but
but wherever
wherever it
it went
went it
it was
was not
not quick
quick enough
enough to
to escape
escape Gandalf.
Gandalf. Pop!
Pop! he
he sent
sent a
a smaller
smaller smoke-ring
smoke-ring from
from his
his short
short clay-pipe
clay-pipe straight
straight through
through each
each one
one of
of Thorin’s.
Thorin’s. Then
Then Gandalfs
Gandalfs 

went down
down a
a rabbit-hole,
rabbit-hole, and
and in
in this
this way
way the
the battle
battle was
was won
won and
and the
the game
game of
of Golf
Golf invented
invented at
at the
the same
same moment.
moment. In
In the
the meanwhile,
meanwhile, however,
however, Bullroarer’s
Bullroarer’s gentler
gentler descendant
descendant was
was reviving
reviving in
in the
the drawing-
drawing- room.
room. After
After a
a while
while and
and a
a drink
drink he
he crept
crept nervously
nervously to
to the
the door
door of
of the
the parlour.
parlour. This
This is
is what
what he
he heard,
heard, Gloin
Gloin speaking:
speaking: “Humph!”
“Humph!” (or
(or some
some snort
snort more
more or
or less
less like
like that).
that). “Will
“Will he
he do,
do, do
do you
you think?
think? It
It is
is all
all very
very well
well for
for Gandalf
Gandalf to
to talk
talk about
about this
this hobbit
hobbit being
being fierce,
fierce, but
but one
one shriek
shriek like
like that
that in
in a
a moment
moment of


...

“Moria!” and
and “Dain,
“Dain, Dain!”
Dain!” the
the dwarves
dwarves of
of the
the Iron
Iron Hills
Hills plunged
plunged in,
in, wielding
wielding their
their mattocks,
mattocks, upon
upon the
the other
other side;
side; and
and beside
beside them
them came
came the
the men
men of
of the
the Lake
Lake with
with long
long swords.
swords. Panic
Panic came
came upon
upon the
the Goblins;
Goblins; and
and even
even as
as they
they turned
turned to
to meet
meet this
this new
new attack,
attack, the
the elves
elves charged
charged again
again with
with renewed
renewed numbers.
numbers. Already
Already many
many of
of the
the goblins
goblins were
were flying
flying back
back down
down the
the river
river to
to escape
escape from
from the
the trap;
trap; and
and many
many of
of their
their own
own wolves
wolves were
were turning
turning upon
upon them
them and
and rending
rending the
the dead
dead and
and the
the wounded.
wounded. Victory
Victory seemed
seemed at
at hand,
hand, when
when a
a c

the host.
host. When
When Gandalf
Gandalf saw
saw Bilbo,
Bilbo, he
he was
was delighted.
delighted. “Baggins!”
“Baggins!” he
he exclaimed.
exclaimed. “Well
“Well I
I never!
never! Alive
Alive after
after all
all —
— I
I am
am glad!
glad! I
I began
began to
to wonder
wonder if
if even
even your
your luck
luck would
would see
see you
you through!
through! A
A terrible
terrible business,
business, and
and it
it nearly
nearly was
was disastrous.
disastrous. But
But other
other news
news can
can wait.
wait. Come!”
Come!” he
he said
said more
more gravely.
gravely. “You
“You are
are called
called for;”
for;” and
and leading
leading the
the hobbit
hobbit he
he took
took him
him within
within the
the tent.
tent. “Hail!
“Hail! Thorin,”
Thorin,” he
he said
said as
as he
he entered.
entered. “I
“I have
have brought
brought him.”
him.” There
There indeed
indeed lay
lay Thorin
Thorin Oakenshield,
Oakenshield, wounded
wounded with
with many
many wounds,
wounds, and
and his
his rent
rent armour
armou

Mountain he
he added:
added: “Farewell
“Farewell Thorin
Thorin Oakenshield!
Oakenshield! And
And Fili
Fili and
and Kili!
Kili! May
May your
your memory
memory never
never fade!”
fade!” Then
Then the
the dwarves
dwarves bowed
bowed low
low before
before their
their Gate,
Gate, but
but words
words stuck
stuck in
in their
their throats.
throats. “Good-
“Good- bye
bye and
and good
good luck,
luck, wherever
wherever you
you fare!”
fare!” said
said Balin
Balin at
at last.
last. “If
“If ever
ever you
you visit
visit us
us again,
again, when
when our
our halls
halls are
are made
made fair
fair once
once more,
more, then
then the
the feast
feast shall
shall indeed
indeed be
be splendid!”
splendid!” “If
“If ever
ever you
you are
are passing
passing my
my way,”
way,” said
said Bilbo,
Bilbo, “don’t
“don’t wait
wait to
to knock!
knock! Tea
Tea is
is at
at four;
four; but
but any
any of
of you
you are
are welcome
welcome at
at anytime!”
anytime!” Then
Then he
he turned
turned away.
away. The
The elf

Lullaby! Alder
Alder and
and Willow/
Willow/ Sigh
Sigh no
no more
more Pine,
Pine, till
till the
the wind
wind of
of the
the mom!
mom! Fall
Fall Moon!
Moon! Dark
Dark be
be the
the land!
land! Hush!
Hush! Hush!
Hush! Oak,
Oak, Ash,
Ash, and
and Thom!
Thom! Hushed
Hushed be
be all
all water,
water, till
till dawn
dawn is
is at
at hand!
hand! “Well,
“Well, Merry
Merry People!”
People!” said
said Bilbo
Bilbo looking
looking out.
out. “What
“What time
time by
by the
the moon
moon is
is this?
this? Your
Your lullaby
lullaby would
would waken
waken a
a drunken
drunken goblin!
goblin! Yet
Yet I
I thank
thank you.”
you.” “And
“And your
your snores
snores would
would waken
waken a
a stone
stone dragon
dragon —
— yet
yet we
we thank
thank you,”
you,” they
they answered
answered with
with laughter.
laughter. “It
“It is
is drawing
drawing towards
towards dawn,
dawn, and
and you
you have
have slept
slept now
now since
since the
the night’s
night’s beginning.
beginning. Tomorrow,
Tomorrow, perhaps,


of their
their times
times together,
together, of
of course,
course, and
and Bilbo
Bilbo asked
asked how
how things
things were
were going
going in
in the
the lands
lands of
of the
the Mountain.
Mountain. It
It seemed
seemed they
they were
were going
going very
very well.
well. Bard
Bard had
had rebuilt
rebuilt the
the town
town in
in Dale
Dale and
and men
men had
had gathered
gathered to
to him
him from
from the
the Lake
Lake and
and from
from South
South and
and West,
West, and
and all
all the
the valley
valley had
had become
become tilled
tilled again
again and
and rich,
rich, and
and the
the desolation
desolation was
was now
now filled
filled with
with birds
birds and
and blossoms
blossoms in
in spring
spring and
and fruit
fruit and
and feasting
feasting in
in autumn.
autumn. And
And Lake-town
Lake-town was
was refounded
refounded and
and was
was more
more prosperous
prosperous than
than ever,
ever, and
and much
much wealth
wealth went
went up
up and
and down
down the
the Running
R

But remember we don't want to deal with words as strings. We need the indexes. And we need the words we decided to skip (the singletons) to be replaced with the index for UNK.

In [199]:
bigrams = [ (word2index[words[i-1]], word2index[words[i]]) for i in range(1, len(words)) ]

In [200]:
bigramFreqs = Counter(bigrams)

In [201]:
len(bigrams)

95939

In [146]:
bigramFreqs.items()

dict_items([((0, 0), 651), ((0, 1), 2), ((1, 0), 4), ((0, 2), 4), ((2, 3), 1), ((3, 0), 33), ((0, 4), 28), ((4, 5), 7), ((5, 6), 5), ((6, 7), 6), ((7, 8), 527), ((8, 9), 20), ((9, 10), 1), ((10, 11), 2), ((11, 5), 2), ((5, 12), 1), ((12, 13), 1), ((13, 5), 3), ((5, 14), 1), ((14, 0), 1), ((0, 15), 1), ((15, 16), 1), ((16, 17), 1), ((17, 18), 26), ((18, 8), 88), ((8, 0), 717), ((0, 19), 241), ((19, 0), 222), ((0, 20), 850), ((20, 21), 8), ((21, 0), 30), ((0, 22), 1), ((22, 23), 1), ((23, 24), 1), ((24, 5), 1), ((5, 25), 1), ((25, 0), 1), ((0, 6), 2), ((6, 18), 1), ((18, 26), 2), ((26, 7), 2), ((7, 27), 3), ((27, 28), 16), ((28, 29), 8), ((29, 30), 4), ((30, 31), 12), ((31, 32), 1), ((32, 28), 16), ((28, 0), 172), ((0, 27), 48), ((27, 33), 159), ((33, 5), 127), ((5, 34), 1), ((34, 20), 2), ((20, 35), 40), ((35, 36), 1), ((36, 0), 1), ((0, 37), 56), ((37, 38), 8), ((38, 5), 49), ((5, 39), 2), ((39, 40), 1), ((40, 41), 1), ((41, 42), 2), ((42, 5), 56), ((5, 0), 312), ((0, 43), 1), ((43, 44

In [202]:
bigramFreqs[(word2index["of"], word2index["the"])]

809

In [203]:
bigramFreqs[(word2index["of"], word2index["Ralphie"])]

222

In [204]:
bigramFreqs[(word2index["of"],0)]

222

In [205]:
unigrams = [word2index[word] for word in words]

In [111]:
unigrams

[1,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 3,
 4,
 0,
 0,
 0,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 6,
 13,
 14,
 6,
 15,
 0,
 16,
 17,
 18,
 19,
 9,
 0,
 20,
 0,
 21,
 22,
 0,
 23,
 24,
 25,
 6,
 26,
 0,
 0,
 7,
 19,
 27,
 8,
 28,
 29,
 30,
 31,
 32,
 33,
 29,
 0,
 28,
 34,
 6,
 35,
 21,
 36,
 37,
 0,
 38,
 39,
 6,
 40,
 41,
 42,
 43,
 6,
 0,
 44,
 45,
 19,
 6,
 0,
 46,
 47,
 0,
 8,
 9,
 48,
 49,
 50,
 42,
 51,
 32,
 29,
 6,
 0,
 52,
 43,
 6,
 0,
 6,
 53,
 54,
 55,
 56,
 57,
 19,
 0,
 58,
 21,
 0,
 0,
 21,
 0,
 59,
 19,
 0,
 0,
 21,
 60,
 21,
 60,
 20,
 0,
 61,
 0,
 21,
 62,
 63,
 9,
 64,
 34,
 65,
 20,
 0,
 50,
 55,
 66,
 32,
 21,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 9,
 75,
 20,
 9,
 76,
 63,
 50,
 77,
 78,
 79,
 9,
 80,
 61,
 81,
 82,
 41,
 83,
 28,
 63,
 21,
 81,
 84,
 41,
 85,
 51,
 86,
 20,
 87,
 88,
 32,
 89,
 75,
 21,
 90,
 32,
 91,
 92,
 68,
 0,
 61,
 9,
 0,
 0,
 0,
 93,
 0,
 0,
 20,
 0,
 0,
 94,
 39,
 95,
 96,
 0,
 29,
 0,
 0,
 0,
 79,
 97,
 32,
 9,
 98,
 99,
 21,
 100,
 32,
 9,

In [206]:
unigramFreqs = Counter(unigrams)

In [207]:
def unigramProb(unigram): 
    return unigramFreqs[unigram]/(sum(unigramFreqs.values()))

In [208]:
word2index["Gandalf"]

363

In [209]:
unigramProb(363)

0.0010318949343339587

In [210]:
unigramFreqs[363]

99

In [211]:
test1 = "In a hole in the ground there lived a hobbit"

In [213]:
def unigramSentenceProb(words): 
    return np.prod([unigramProb(word2index[word]) for word in words])


In [214]:
unigramSentenceProb(test1.split())

7.5075513010812298e-27

In [215]:
test1.split()

['In', 'a', 'hole', 'in', 'the', 'ground', 'there', 'lived', 'a', 'hobbit']

In [216]:
[word2index[word] for word in test1.split()]

[5, 6, 7, 8, 9, 10, 11, 12, 6, 64]

For a bigram model we need the product of the bigram probs in a sentence. For bigram probs we need the bigram count divided by the unigram count of the first word.

In [217]:
def bigramProb(bigram):
    return (bigramFreqs[bigram]/unigramFreqs[bigram[0]])

In [218]:
def bigramSentenceProb(words):
    return np.prod([bigramProb((word2index[words[i-1]], word2index[words[i]])) for i in range(1, len(words))])

In [219]:
bigramProb((word2index["of"], word2index["the"]))

0.3293973941368078

In [220]:
bigramSentenceProb(test1.split())

5.4976256439726721e-15